In [38]:
import requests
import pandas as pd
import re
import numpy as np
import json

In [39]:
# 任意两点间的路径
pg_url = 'http://service.shmetro.com/i/c?o=' + '0120' + '&d=' + '0825' + '&t=0'
return_data = requests.get(pg_url)
data = return_data.json()
number_path = data['data']['total']  # 给出了几条路径
all_path = pd.DataFrame()
for jj in range(0, number_path):
	route_detail = data['data']['list'][jj]
	tem = pd.DataFrame([route_detail])
	# 计算换乘时间
	pathidt = pd.DataFrame({'PSID': tem['psid'].str.split('-')[0], 'PSTM': tem['pstm'].str.split('-')[0]})
	tempath = pd.DataFrame(re.split('[()-]', tem['ptid'][0]))
	tempath = tempath[tempath[0] != '']
	nedd = pathidt['PSTM'][pathidt['PSID'].isin(tempath[0])].reset_index(drop=True)
	hct = np.sum(nedd[1::2].astype(int).values - nedd[::2].astype(int).values)
	tem['huanchentime'] = hct
	all_path = all_path.append(tem)
# 任意两点间的票价
pg_url = 'http://service.shmetro.com/i/p?o=' + '0120' + '&d=' + '0825' + '&t=0'
return_data = requests.get(pg_url)
data = return_data.json()
all_path['price'] = data['data']['p']


def getPath(oid, did):
    pg_url = 'http://service.shmetro.com/i/c?o=' + oid + '&d=' + did + '&t=0'
    return_data = requests.get(pg_url)
    data = return_data.json()
    number_path = data['data']['total']  # 给出了几条路径
    all_path = pd.DataFrame()
    for jj in range(0, number_path):
        route_detail = data['data']['list'][jj]
        tem = pd.DataFrame([route_detail])
        # 计算换乘时间
        pathidt = pd.DataFrame({'PSID': tem['psid'].str.split('-')[0], 'PSTM': tem['pstm'].str.split('-')[0]})
        tempath = pd.DataFrame(re.split('[()-]', tem['ptid'][0]))
        tempath = tempath[tempath[0] != '']
        nedd = pathidt['PSTM'][pathidt['PSID'].isin(tempath[0])].reset_index(drop=True)
        hct = np.sum(nedd[1::2].astype(int).values - nedd[::2].astype(int).values)
        tem['huanchentime'] = hct
        all_path = all_path.append(tem)
    # 任意两点间的票价
    pg_url = 'http://service.shmetro.com/i/p?o=' + oid + '&d=' + did + '&t=0'
    return_data = requests.get(pg_url)
    data = return_data.json()
    all_path['price'] = data['data']['p']
    return all_path
all_path

,d,no,psnm,pttm,obegintime,olasttime,o,psid,costtime,lttm,ptid,lstm,pstm,plid,bstm,begintraveltime,passengerscale,huanchentime,price
0,0825,1,11,1242,21718,82089,0120,120-740-741-742-743-744-745-746-829-828-827-82...,1909,2032,746(829),,0-0-174-303-424-637-794-922-1242-1370-1524-170...,7-8,,1691,0.6400,320,4
0,0825,2,8,909-1965,20539,81862,0120,120-119-118-1149-1150-1151-1152-1153-1154-826-825,2173,2259,118(1149)-1154(826),,0-122-259-909-1084-1239-1364-1481-1665-1965-2173,1-11-8,,4010,0.2200,950,4
0,0825,3,13,746,18419,81991,0120,120-121-122-123-835-834-833-832-831-830-829-82...,2221,2130,123(835),,0-111-237-396-746-864-993-1110-1247-1425-1554-...,1-8,,4502,0.1400,350,4


In [40]:
with open("../data/metro_data.json",'r', encoding='utf-8') as load_f:
    metro_data = json.load(load_f)
    # print(metro_data)
    lines = metro_data['lines']
    stations = metro_data['stations']

def getStaByNameMy(name):
    for sta in stations:
        if sta['name'].rstrip()==name:
            return sta
    print(f'{name}不存在')
    return None

def getStaByIdMy(key):
    for sta in stations:
        if sta['id'].rstrip()==key:
            return sta
    print(f'{key}不存在')
    return None

In [41]:
with open("../data/sta_official.json", 'r',encoding='utf-8') as load_f:
    sta_official = json.load(load_f)
# sta_official

def getStaByName(name):
    for sta in sta_official:
        if sta['value'].rstrip()==name:
            return sta
    print(f'{name}不存在')
    return None
def getStaById(key):
    for sta in sta_official:
        if sta['key'].rstrip()==key:
            return sta
    print(f'{key}不存在')
    return None

In [37]:
path = []
def toMyPath(path_item):
    ret = {}
    
for station_o in stations:
    sta_off_o = getStaByName(station_o['name'])
    for station_d in stations:
        sta_off_d = getStaByName(station_d['name'])
        path_res = getPath(sta_off_o['key'], sta_off_d['key'])
        path_item = {}
        path_item['od'] = {'o_sid':station_o['id'], 'd_sid':station_d['id']}
        path_item['paths'] = []
        for 